In [8]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from sqlalchemy import create_engine
import pandas as pd
import json
import requests
import io

In [9]:
# Read the JSON config file
config_file_path = 'config.json'
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file) 

# Get the connection string
CONNECTION_STRING = config['AZURE_CONNECTION_STRING']

In [10]:
def upload_to_azure(data, blob_name, container_name):
    """
    This function uploads the data to azure blob storage
    data: data to be uploaded
    blob_name: name of the blob
    container_name: name of the container in the storage of the resource group 
    """
    blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data.getvalue(), overwrite=True)

In [11]:
def get_azure_blob(container_name):
    blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)
    container_client = blob_service_client.get_container_client(container_name)
    blob_list = container_client.list_blobs()
    return blob_list

In [12]:
def get_azure_blob_data(container_name, blob):
    blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob.name)
    blob_content = blob_client.download_blob().readall()
    return blob_content

In [13]:
def download_file(url):
    response = requests.get(url)
    return io.BytesIO(response.content)

In [14]:
def transform_ppp_loan_data():
    container_name = 'pppdata'
    blob_list = get_azure_blob(container_name)
    print(f"Found blobs in the container")
    for blob in blob_list:
        if "public_150k_plus" in blob.name:
            print(f"Downloading {blob.name}")
            blob_data = get_azure_blob_data(container_name, blob)
            print(f"Downloaded {blob.name} successfully")
            data = io.BytesIO(blob_data)
            print(f"Reading {blob.name}")
            df = pd.read_csv(data)
            return df


In [15]:
def transform_naics_data():
    return None

In [16]:
def transform_gdp_data():
    return None

In [22]:
df = transform_ppp_loan_data()

#Use factorize to numerically encode the different business types
#Example: 'Corporation' -> 0, 'Non-Profit Organization' -> 1, 'Sole Proprietorship' -> 2
df_business_type = df['BusinessType'].unique()
dim_business_type = pd.DataFrame(df_business_type, columns=['BusinessType'])

dim_business_type['business_type_id'] = pd.factorize(dim_business_type['BusinessType'])[0]
dim_business_type

Found blobs in the container
Downloaded public_150k_plus_230930.csv successfully
Reading public_150k_plus_230930.csv


,BusinessType,business_type_id
0,Corporation,0
1,Sole Proprietorship,1
2,Non-Profit Organization,2
3,NaN,-1
4,Subchapter S Corporation,3
5,Limited Liability Company(LLC),4
6,501(c)3 – Non Profit,5
7,Cooperative,6
8,Partnership,7
9,Professional Association,8


In [18]:
if __name__ == "__main__":
    # Get the head of the data
    #transform_gdp_data()
    #transform_naics_data()
    #transform_gdp_data()
    pass

SyntaxError: incomplete input (1189515278.py, line 5)